In [1]:
PYTHON_PATHS = [".."]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)

In [2]:
import numpy as np
import os
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt

In [3]:
from unsupkeypoints.data import DescriptorDataModule
from unsupkeypoints.models import PointNetwork
from unsupkeypoints.criterions import RegressionLoss
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.parsing import AttributeDict
import torch
import pytorch_lightning as pl

# Data module

In [4]:
ROOT_PATH = os.path.dirname(os.getcwd())
TRAIN_PATH = os.path.join(ROOT_PATH, "data/output_kapture/7scenes/fire/mapping")
TEST_PATH = os.path.join(ROOT_PATH, "data/output_kapture/7scenes/fire/query")

In [5]:
data_module = DescriptorDataModule(TRAIN_PATH, TEST_PATH)

[DescriptorDataModule] - train dataset size 568196
[DescriptorDataModule] - test dataset size 544913


# Model

In [6]:
params = AttributeDict(
    name="point_net",
    optimizer=AttributeDict(),
    hidden_dimensions=(100,),
    input_dimension=512,
)

In [7]:
criterion = RegressionLoss()
model = PointNetwork(params, criterion)
data_module._batch_size = 512

In [8]:
trainer = pl.Trainer(logger=TensorBoardLogger("lightning_logs"), max_epochs=10, gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [9]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type           | Params
----------------------------------------------
0 | _criterion | RegressionLoss | 0     
1 | backend    | Sequential     | 51.6 K
----------------------------------------------
51.6 K    Trainable params
0         Non-trainable params
51.6 K    Total params
0.206     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:610: LightningDeprecationWarning: Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.
  warning_cache.deprecation(


Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/trainer.py:897: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn('Detected KeyboardInterrupt, attempting graceful shutdown...')


In [10]:
trainer.test(model, data_module.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'median_position_error': 0.6074175624252494,
 'median_rotation_error': 19.572889225745893,
 'point_count': 2000.0,
 'reconstruction_median_position_error': 0.004407721311686942,
 'reconstruction_median_rotation_error': 0.17272340737539726,
 'test_loss': 0.15257149934768677,
 'test_mean_dist': 0.3133174479007721}
--------------------------------------------------------------------------------


[{'test_loss': 0.15257149934768677,
  'test_mean_dist': 0.3133174479007721,
  'median_position_error': 0.6074175624252494,
  'median_rotation_error': 19.572889225745893,
  'reconstruction_median_position_error': 0.004407721311686942,
  'reconstruction_median_rotation_error': 0.17272340737539726,
  'point_count': 2000.0}]